In [1]:
import pandas as pd
import numpy as np
#import datetime
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

## Funzioni accessorie

In [2]:
def moving_function(data, fun, left_win, right_win):
    '''Applica una funzione mobile su un array rispetto alle ampiezze indicate
    per le finestre a sx e dx
    '''
    out = np.zeros_like(data)
    num_samples = len(data)
    lefts = list(map(lambda x: x-left_win, list(range(num_samples))))
    rights = list(map(lambda x: x+right_win+1, list(range(num_samples))))
    for i in range(num_samples):
        out[i] = fun(data[max(0, lefts[i]):min(num_samples, rights[i])])
    return out


## Parametri

In [3]:
data_file = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'


lisciatura=3

## Lettura dati e preprocessing

In [4]:
data=pd.read_csv(data_file)
data['data'].apply(lambda x: x[0:10])

giorni=pd.unique(data['data'])
day_counter=list(range(len(giorni)))
print('Giorni osservati: %d'%len(giorni))
print('Da: %s'%giorni[0][0:10])
print(' A: %s'%giorni[-1][0:10])
regioni=pd.unique(data['denominazione_regione'])

for colname in data.columns:
    print(colname,end=', ')


Giorni osservati: 689
Da: 2020-02-24
 A: 2022-01-12
data, stato, codice_regione, denominazione_regione, lat, long, ricoverati_con_sintomi, terapia_intensiva, totale_ospedalizzati, isolamento_domiciliare, totale_positivi, variazione_totale_positivi, nuovi_positivi, dimessi_guariti, deceduti, casi_da_sospetto_diagnostico, casi_da_screening, totale_casi, tamponi, casi_testati, note, ingressi_terapia_intensiva, note_test, note_casi, totale_positivi_test_molecolare, totale_positivi_test_antigenico_rapido, tamponi_test_molecolare, tamponi_test_antigenico_rapido, codice_nuts_1, codice_nuts_2, 

### Formazione dataframe nazionale

In [5]:
data_italia = pd.DataFrame(data.groupby(['data']).sum())
data_italia.reset_index(inplace=True)
data_italia.tail(5)

# aggiunta info da colonne calcolate
data_italia['deceduti_giornalieri'] = data_italia['deceduti'].diff()


## Grafici nazionali

In [6]:
indicatori_selected = ['totale_casi',
                       'nuovi_positivi',
                       'totale_positivi',
                       'deceduti_giornalieri']


In [7]:
for indicatore in indicatori_selected:
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data_italia['data'],
                             y=data_italia[indicatore],
                             mode='markers'))
    # versione lisciata
    fig.add_trace(go.Scatter(x=data_italia['data'],
                             y=moving_function(data_italia[indicatore],np.mean,lisciatura,lisciatura),
                             mode='lines'))

    fig.update_layout(
        title='%s, Italia' %indicatore,
        autosize=False,
        showlegend=False,
        width=900,
        height=600)
    fig.show()
